In [ ]:
# Linear versus quadratic model estimation
# hollander et al., 2021
# test for decoding accuracy
# increase linearly or show a clear peak at a specific cortical depth
# perform bayesian model comparison

# fit two hierarhical linear models using pymc
# NUTS sampler

# dependent variable (decoding accuracy) was first modeled as a linear function of cortical depth
# y_n = theta_n,0 + theta_n,1 * d_n

# and then also as a quadratic function of cortical depth
# y_n = theta_n,0 + theta_n,1 * d_n * theta_n,2 * d^2

# where y_n were all observationf for subjects n
# d is a corresponding vector of cortical depths
# theta_n is the parameter vector for subject n that was estimated

# the subject-wise parameters were modeled as coming from Gaussian group distirbution mean mu and standard deviation sigma
# theta ~ N(mu, sigma)
# mu ~ N(0, 1)
# sigma ~ HalfCauchy(5) -> 5 subjects?

# (1) use the state-of-the-art Watanabe-Akaike informatoin criterion to do Bayesian model copmarison between the linear and quadratic models
# (2) estimate the posterior of the peak of the quadratic function using the formula 

In [6]:
from pathlib import Path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
from src.config import SUBJECTS, SESSION, N_LAYER

def get_profile(subj, sess, day):
    path = Path(DIR_DATA) / subj / f"{sess}{SESSION[subj][sess][day]}"
    file = path / "bandpass_none" / "accuracy.csv"
    data = np.genfromtxt(file, delimiter=',')
    return data

SESS = "GE_EPI"
DIR_DATA = "/data/pt_01880/Experiment1_ODC/paper/decoding"

x = []
y = []
group = []
for i, subj in enumerate(SUBJECTS):
    for day in [0, 1]:
        x.extend(np.reshape(np.repeat(np.arange(N_LAYER),10), (11, 10)).flatten())
        y.extend(get_profile(subj, SESS, day).flatten())
        group.extend(i * np.ones_like(get_profile(subj, SESS, day).flatten(), dtype=np.int64))

In [14]:
import pymc as pm


n_group = 5




with pm.Model():
    # hyperpriors
    # sigma: model the uncertainty of our parent distributions
    mu_theta0 = pm.Normal("mu_theta0", 0, 1)
    mu_theta1 = pm.Normal("mu_theta1", 0, 1)
    sigma_theta0 = pm.HalfCauchy("sigma_theta0", 5)
    sigma_theta1 = pm.HalfCauchy("sigma_theta1", 5)

    theta0 = pm.Gamma("theta0", mu_theta0, sigma_theta0, shape=n_group)
    theta1 = pm.Gamma("theta1", mu_theta1, sigma_theta1, shape=n_group)
    sigma_margin = pm.Exponential("sigma_margin", 0.1)

    y_est = theta0[group] + theta1[group] * x
    y_like = pm.Normal(
        "y_like",
        mu=y_est,
        sigma=sigma_margin,
        observed=y,
    )

    trace = pm.sample(
        draws=10000,
        tune=500,
        chains=5,
        progressbar=True,
        target_accept=0.95,
        return_inferencedata=True,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/data/pt_01880/source/miniconda3/envs/odc_review/lib/python3.10/site-packages/pytensor/tensor/elemwise.py:752: RuntimeWarning: invalid value encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/data/pt_01880/source/miniconda3/envs/odc_review/lib/python3.10/site-packages/pytensor/tensor/elemwise.py:752: RuntimeWarning: invalid value encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/data/pt_01880/source/miniconda3/envs/odc_review/lib/python3.10/site-packages/pytensor/tensor/elemwise.py:752: RuntimeWarning: invalid value encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Multiprocess sampling (5 chains in 4 jobs)
NUTS: [mu_theta0, mu_theta1, sigma_theta0, sigma_theta1, beta, sigma_bold, sigma_margin]


Sampling 5 chains for 500 tune and 10_000 draw iterations (2_500 + 50_000 draws total) took 157 seconds.
There were 69 divergences after tuning. Increase `target_accept` or reparameterize.
